In [1]:
#================================================================
# Evaluator
#================================================================

import re
import string
import unicodedata

def normalize_text(s):
    # Normalize Unicode characters
    s = unicodedata.normalize('NFD', s)
    # Convert to lowercase
    s = s.lower()
    # Remove punctuation
    s = ''.join(ch for ch in s if ch not in string.punctuation)
    # Remove articles (a, an, the)
    s = re.sub(r'\b(a|an|the)\b', ' ', s)
    # Fix extra whitespaces
    s = ' '.join(s.split())
    return s


def f1_score_strings(label, pred):
    # Tokenize the strings (split by whitespace)
    tokens1 = set(normalize_text(label).split())
    tokens2 = set(normalize_text(pred).split())

    # Calculate true positives, false positives, and false negatives
    true_positives = len(tokens1 & tokens2)
    false_positives = len(tokens2 - tokens1)
    false_negatives = len(tokens1 - tokens2)

    if true_positives == 0:
        return 0
    
    # Calculate F1 score
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

In [2]:
#================================================================
# Data Loader
#================================================================

import json

def load_data_minor():
    with open("data._json", "r") as f:
        data = json.load(f)
    # raw format:
    # [
    #     {
    #         "question": "...",
    #         "docs": [...],
    #         "answer": "..."
    #     },
    #     ...
    # ]
          
    # format to (input, output) pairs
    new_data = []
    for d in data:
        input = (d["question"], d["docs"])
        output = d["answer"]
        new_data.append((input, output))
    return new_data[:5], None, new_data[5:]

In [3]:
#================================================================
# Optimizer Set Up
#================================================================

from cognify.optimizer.core import driver, flow
from cognify.cog_hub import reasoning, ensemble
from cognify.cog_hub.common import NoChange
from cognify.cog_hub.fewshot import LMFewShot
from cognify.cog_hub.reasoning import ZeroShotCoT
from cognify.optimizer.control_param import ControlParameter

# ================= Inner Loop Config =================
# Reasoning Parameter
reasoning_param = reasoning.LMReasoning(
    [NoChange(), ZeroShotCoT()] 
)
# Few Shot Parameter
few_shot_params = LMFewShot(2)

# Layer Config
inner_opt_config = flow.OptConfig(
    n_trials=2,
)
inner_loop_config = driver.LayerConfig(
    layer_name='inner_loop',
    universal_params=[few_shot_params, reasoning_param],
    opt_config=inner_opt_config,
)

# ================= Outer Loop Config =================
# Ensemble Parameter
general_usc_ensemble = ensemble.UniversalSelfConsistency(3)
general_ensemble_params = ensemble.ModuleEnsemble(
    [NoChange(), general_usc_ensemble]
)
# Layer Config
outer_opt_config = flow.OptConfig(
    n_trials=2,
)
outer_loop_config = driver.LayerConfig(
    layer_name='outer_loop',
    universal_params=[general_ensemble_params],
    opt_config=outer_opt_config,
)

# ================= Overall Control Parameter =================
optimize_control_param = ControlParameter(
    opt_layer_configs=[outer_loop_config, inner_loop_config],
    opt_history_log_dir='opt_results',
    evaluator_batch_size=2,
)

In [4]:
train, val, test = load_data_minor()

In [5]:
import cognify.run.optimize

opt_cost, pareto_frontier, opt_logs = cognify.run.optimize.optimize(
    script_path="cognify_workflow.py",
    control_param=optimize_control_param,
    train_set=train,
    val_set=val,
    eval_fn=f1_score_strings,
)

> Evaluation in dry_run | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<?, ?it/s]

> outer_loop | (best score: 0.00, lowest cost@1000: 0.00 $):   0%|          | 0/2 [00:00<?, ?it/s]

------> Evaluation in outer_loop_0 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<?, …

------> Evaluation in outer_loop_1 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<?, …

---> inner_loop in outer_loop_0 | (best score: 0.00, lowest cost@1000: 0.00 $):   0%|          | 0/2 [00:00<?,…

...

------> Evaluation in outer_loop_0.inner_loop_0 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0…

------> Evaluation in outer_loop_1.inner_loop_0 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0…

------> Evaluation in outer_loop_1.inner_loop_1 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0…

[INFO 2024-11-14 05:09:02] ----------------- Optimization Finished -----------------


=========== Optimization Results ===========
Num Pareto Frontier: 1
--------------------------------------------------------
Pareto_1
  Quality: 0.402, Cost per 1K invocation ($): 0.17
  Applied Optimization: outer_loop_0.inner_loop_1


In [6]:
from cognify.run.evaluate import evaluate, load_workflow

In [7]:
new_workflow = load_workflow(control_param=optimize_control_param, config_id='Pareto_1')

In [8]:
input = (
    "What was the 2010 population of the birthplace of Gerard Piel?", 
    [
        'Gerard Piel | Gerard Piel (1 March 1915 in Woodmere, N.Y. – 5 September 2004) was the publisher of the new Scientific American magazine starting in 1948. He wrote for magazines, including "The Nation", and published books on science for the general public. In 1990, Piel was presented with the "In Praise of Reason" award by the Committee for Skeptical Inquiry (CSICOP).',
        'Woodmere, New York | Woodmere is a hamlet and census-designated place (CDP) in Nassau County, New York, United States. The population was 17,121 at the 2010 census.',
    ],
)

new_workflow(input)

'The population of Woodmere, New York in 2010 was 17,121.'

In [9]:
eval_result = evaluate(
    control_param=optimize_control_param,
    config_id='Pareto_1',
    test_set=test,
    n_parallel=10,
    eval_fn=f1_score_strings,
    save_to='eval_results.json',
)

----- Testing select trial outer_loop_0.inner_loop_1 -----
  Params: {'qa_agent_few_shot': 'NoChange', 'qa_agent_reasoning': 'NoChange'}
  Quality: 0.402, Cost per 1K invocation ($): 0.17 $


> Evaluation in outer_loop_0.inner_loop_1 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/10 [0…

=========== Evaluation Results ===========
  Quality: 0.453, Cost per 1K invocation ($): 0.16 $
